# From this repo - https://github.com/joaomdmoura/crewAI-examples/tree/main/CrewAI-LangGraph

In [88]:
%pip install crewai langgraph --quiet

Note: you may need to restart the kernel to use updated packages.


In [89]:
%pip install python-dotenv --quiet
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [90]:
from langgraph.graph import StateGraph

from typing import TypedDict, Optional

class BashScriptState(TypedDict):
	code: Optional[str] = None
	valid_markdown: bool
	valid_bash: bool 

In [91]:

workflow = StateGraph(BashScriptState)

def generate_bash_code_as_markdown_node(state: BashScriptState):
	state["code"] = """
	```bash
	ls
	```
	"""
	return state

def check_markdown_syntax_node(state: BashScriptState):
	state["valid_markdown"] = True
	return state

def check_bash_syntax_node(state: BashScriptState):
	state["valid_bash"] = True
	return state


def improve_code(state: BashScriptState):
	return state

def deploy_code(state: BashScriptState):
	print("Deploying code successfully")
	return state

def approve_code(state: BashScriptState):
	print(state)
	return "valid"

workflow.add_node("generate_bash_code_as_markdown", generate_bash_code_as_markdown_node)
workflow.add_node("check_markdown_syntax", check_markdown_syntax_node)
workflow.add_node("check_bash_syntax", check_bash_syntax_node)
#workflow.add_node("improve_code", DevOpsCrew().kickoff)

workflow.add_node("improve_code",improve_code)
workflow.add_node("deploy_code", deploy_code)

workflow.set_entry_point("generate_bash_code_as_markdown")

workflow.add_conditional_edges(
    "generate_bash_code_as_markdown",
	approve_code,
	{
		"valid": "check_markdown_syntax",
		"non_valid": "improve_code"
	}
)

workflow.add_conditional_edges(
	"check_markdown_syntax",
	approve_code,
	{
		"valid": "check_bash_syntax",
		"non_valid": "improve_code"
	}
)


workflow.add_conditional_edges(
	"check_bash_syntax",
	approve_code,
	{
		"valid": "deploy_code",
		"non_valid": "improve_code"
	}
)

workflow.add_edge("improve_code", "check_markdown_syntax")
workflow.set_finish_point('deploy_code')

app = workflow.compile()
inputs = {"question": "Generate a bash script as markdown that list the files in the current directory"}
result = app.invoke(inputs)
print(result)

{'code': '\n\t```bash\n\tls\n\t```\n\t', 'valid_markdown': None, 'valid_bash': None}
{'code': '\n\t```bash\n\tls\n\t```\n\t', 'valid_markdown': True, 'valid_bash': None}
{'code': '\n\t```bash\n\tls\n\t```\n\t', 'valid_markdown': True, 'valid_bash': True}
Deploying code successfully
{'code': '\n\t```bash\n\tls\n\t```\n\t', 'valid_markdown': True, 'valid_bash': True}
